In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import dill

from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier

from sklearn.feature_selection import SelectKBest, f_classif, chi2, SelectFromModel
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline        
from sklearn.model_selection import cross_val_score,cross_validate
from sklearn.linear_model import LogisticRegression

%matplotlib inline
SEED=50

In [2]:
cns = pd.read_csv('cns.csv')
cns.head()

,x.AFFX.BioB.5.at,x.AFFX.BioB.M.at,x.AFFX.BioB.3.at,x.AFFX.BioC.5.at,x.AFFX.BioC.3.at,x.AFFX.BioDn.5.at,x.AFFX.BioDn.3.at,x.AFFX.CreX.5.at,x.AFFX.CreX.3.at,x.AFFX.BioB.5.st,...,x.U58516.at,x.U73738.at,x.X06956.at,x.X16699.at,x.X83863.at,x.Z17240.at,x.L49218.f.at,x.M71243.f.at,x.Z78285.f.at,y
1,-0.231140,-0.242715,-0.183060,-0.041937,-0.479553,-0.394078,0.039086,-0.291240,-0.228024,-0.100701,...,-0.050841,-0.309048,-0.261858,-0.197752,0.110761,0.069804,-0.193300,0.013711,-0.199087,2
2,-0.217881,-0.327010,-0.219228,-0.087645,-0.463983,-0.511138,0.144985,-0.428505,-0.194977,-0.262790,...,0.464738,-0.199019,0.025077,-0.254707,0.207858,-0.117734,-0.102015,0.096483,-0.191834,2
3,-0.308496,-0.256856,-0.299157,-0.197525,-0.239276,-0.283775,-0.120614,-0.360136,-0.140941,-0.166761,...,-0.004149,-0.084356,-0.233233,-0.227740,0.065071,-0.173902,-0.274436,-0.159619,-0.239276,2
4,-0.262352,-0.259978,-0.154330,-0.249294,-0.364042,-0.706703,-0.229510,-0.338323,-0.193107,-0.089043,...,0.463725,0.010669,-0.253647,-0.255625,0.405164,-0.132568,-0.175697,0.021353,-0.223575,2
5,-0.274458,-0.154017,-0.061023,-0.098712,-0.452661,-0.454710,-0.274458,-0.367451,-0.183922,-0.108544,...,0.068021,0.089733,-0.383428,-0.210960,0.016403,-0.112231,-0.308050,0.056141,-0.375645,2


In [3]:
print(cns.isnull().values.any())


False


In [4]:
features=cns.iloc[:,0:-1]
labels=cns.iloc[:,-1]

In [5]:
# Estandarizar

min_max_scaler = preprocessing.MinMaxScaler()
X = min_max_scaler.fit_transform(features)

In [6]:
X

array([[0.76977144, 0.72460051, 0.73475917, ..., 0.82484924, 0.86478722,
        0.83183692],
       [0.79632123, 0.51901313, 0.65615052, ..., 1.        , 1.        ,
        0.8467659 ],
       [0.61487744, 0.69011213, 0.48243042, ..., 0.66917029, 0.58164346,
        0.74911813],
       ...,
       [0.43085597, 0.3112476 , 0.16300848, ..., 0.2510696 , 0.03330701,
        0.22218608],
       [0.58932403, 0.44619507, 0.72462594, ..., 0.9068894 , 0.85566845,
        0.85846072],
       [0.50712403, 0.40712679, 0.52396387, ..., 0.85344729, 0.83220874,
        0.80065326]])

In [7]:
y = np.transpose(np.asarray(labels.values - 1, dtype=int))


In [8]:
y

array([1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1,
       0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1])

# 1. SVC

In [9]:
%%time
clf_svc = SVC(kernel='linear', C=1, probability=False, random_state=SEED) 
scores_clf_svc = cross_validate(clf_svc, X, y, cv=10, scoring=('accuracy','precision','recall','f1', 'roc_auc'),return_train_score=True)

CPU times: user 7.27 s, sys: 107 ms, total: 7.38 s
Wall time: 1.85 s


In [10]:
print('Accuracy:', np.mean(scores_clf_svc['test_accuracy']))
print('Precision:', np.mean(scores_clf_svc['test_precision']))
print('Recall:', np.mean(scores_clf_svc['test_recall']))
print('F1:', np.mean(scores_clf_svc['test_f1']))
print('ROC-AUC:', np.mean(scores_clf_svc['test_roc_auc']))

Accuracy: 0.679047619047619
Precision: 0.78
Recall: 0.7916666666666667
F1: 0.7568253968253968
ROC-AUC: 0.6208333333333333


# 2. NAIVE BAYES

In [11]:
%%time
clf_nb = GaussianNB()
scores_clf_nb = cross_validate(clf_nb, X, y, cv=10, scoring=('accuracy','precision','recall','f1','roc_auc'),return_train_score=True)


CPU times: user 659 ms, sys: 28 ms, total: 687 ms
Wall time: 346 ms


/home/nacho/anaconda3/envs/tfm/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/nacho/anaconda3/envs/tfm/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [12]:
print('Accuracy:', np.mean(scores_clf_nb['test_accuracy']))
print('Precision:', np.mean(scores_clf_nb['test_precision']))
print('Recall:', np.mean(scores_clf_nb['test_recall']))
print('F1:', np.mean(scores_clf_nb['test_f1']))
print('ROC-AUC:', np.mean(scores_clf_nb['test_roc_auc']))

Accuracy: 0.6647619047619047
Precision: 0.6633333333333333
Recall: 0.8
F1: 0.7151587301587301
ROC-AUC: 0.6479166666666667


# 3. RANDOM FORESTS

In [13]:
%%time
clf_rf = RandomForestClassifier(n_estimators=100, n_jobs=-1, class_weight='balanced', random_state=SEED)
scores_clf_rf = cross_validate(clf_rf, X, y, cv=10, scoring=('accuracy','precision','recall','f1', 'roc_auc'),return_train_score=True)



CPU times: user 5.14 s, sys: 831 ms, total: 5.98 s
Wall time: 13.1 s


In [14]:
print('Accuracy:', np.mean(scores_clf_rf['test_accuracy']))
print('Precision:', np.mean(scores_clf_rf['test_precision']))
print('Recall:', np.mean(scores_clf_rf['test_recall']))
print('F1:', np.mean(scores_clf_rf['test_f1']))
print('ROC-AUC:', np.mean(scores_clf_rf['test_roc_auc']))

Accuracy: 0.667142857142857
Precision: 0.6904761904761905
Recall: 0.95
F1: 0.788997113997114
ROC-AUC: 0.6


## SELECCIONAR VARIABLES USANDO PUNTUACION F

In [15]:
feat_sel_f = SelectKBest(f_classif, k=10) 



In [16]:
pipe_f_svc = Pipeline([('F Score',feat_sel_f), ('SVM',clf_svc)])


In [17]:
%%time

scores_f_svc = cross_validate(pipe_f_svc, X, y, cv=10, scoring=('accuracy','precision','recall','f1', 'roc_auc'),return_train_score=True)

CPU times: user 245 ms, sys: 36.2 ms, total: 281 ms
Wall time: 280 ms


In [18]:
print('Accuracy SVC:', np.mean(scores_f_svc['test_accuracy']))
print('Precision SVC:', np.mean(scores_f_svc['test_precision']))
print('Recall SVC:', np.mean(scores_f_svc['test_recall']))
print('F1 SVC:', np.mean(scores_f_svc['test_f1']))
print('ROC-AUC SVC:', np.mean(scores_f_svc['test_roc_auc']))

Accuracy SVC: 0.6614285714285715
Precision SVC: 0.72
Recall SVC: 0.8166666666666667
F1 SVC: 0.7415873015873016
ROC-AUC SVC: 0.6125


In [19]:
pipe_f_nb = Pipeline([('F Score',feat_sel_f), ('NB',clf_nb)])


In [20]:
%%time

scores_f_nb = cross_validate(pipe_f_nb, X, y, cv=10, scoring=('accuracy','precision','recall','f1', 'roc_auc'),return_train_score=True)


CPU times: user 238 ms, sys: 28.2 ms, total: 266 ms
Wall time: 265 ms


In [21]:
print('Accuracy NB:', np.mean(scores_f_nb['test_accuracy']))
print('Precision NB:', np.mean(scores_f_nb['test_precision']))
print('Recall NB:', np.mean(scores_f_nb['test_recall']))
print('F1 NB:', np.mean(scores_f_nb['test_f1']))
print('ROC-AUC NB:', np.mean(scores_f_nb['test_roc_auc']))

Accuracy NB: 0.6671428571428571
Precision NB: 0.7416666666666667
Recall NB: 0.7416666666666667
F1 NB: 0.7250000000000001
ROC-AUC NB: 0.5791666666666667


In [22]:
pipe_f_rf = Pipeline([('F Score',feat_sel_f), ('RF',clf_rf)])


In [23]:
%%time


scores_f_rf = cross_validate(pipe_f_rf, X, y, cv=10, scoring=('accuracy','precision','recall','f1', 'roc_auc'),return_train_score=True)

CPU times: user 4.67 s, sys: 917 ms, total: 5.58 s
Wall time: 12.4 s


In [24]:
print('Accuracy RF:', np.mean(scores_f_rf['test_accuracy']))
print('Precision RF:', np.mean(scores_f_rf['test_precision']))
print('Recall RF:', np.mean(scores_f_rf['test_recall']))
print('F1 RF:', np.mean(scores_f_rf['test_f1']))
print('ROC-AUC RF:', np.mean(scores_f_rf['test_roc_auc']))

Accuracy RF: 0.6304761904761904
Precision RF: 0.6804761904761906
Recall RF: 0.7916666666666667
F1 RF: 0.7170129870129871
ROC-AUC RF: 0.6000000000000001


## SELECCIONAR VARIABLES USANDO Chi-Cuadrado

In [25]:
feat_sel_chi = SelectKBest(chi2, k=10) 



In [26]:
pipe_chi_svc = Pipeline([('Chi-cuadrado',feat_sel_chi), ('SVM',clf_svc)])


In [27]:
%%time

scores_chi_svc = cross_validate(pipe_f_svc, X, y, cv=10, scoring=('accuracy','precision','recall','f1', 'roc_auc'),return_train_score=True)

CPU times: user 246 ms, sys: 40.1 ms, total: 286 ms
Wall time: 285 ms


In [28]:
print('Accuracy SVC:', np.mean(scores_chi_svc['test_accuracy']))
print('Precision SVC:', np.mean(scores_chi_svc['test_precision']))
print('Recall SVC:', np.mean(scores_chi_svc['test_recall']))
print('F1 SVC:', np.mean(scores_chi_svc['test_f1']))
print('ROC-AUC SVC:', np.mean(scores_chi_svc['test_roc_auc']))

Accuracy SVC: 0.6614285714285715
Precision SVC: 0.72
Recall SVC: 0.8166666666666667
F1 SVC: 0.7415873015873016
ROC-AUC SVC: 0.6125


In [29]:
pipe_chi_nb = Pipeline([('Chi-cuadrado',feat_sel_chi), ('NB',clf_nb)])


In [30]:
%%time


scores_chi_nb = cross_validate(pipe_chi_nb, X, y, cv=10, scoring=('accuracy','precision','recall','f1', 'roc_auc'),return_train_score=True)


CPU times: user 870 ms, sys: 4.23 ms, total: 875 ms
Wall time: 226 ms


In [31]:
print('Accuracy NB:', np.mean(scores_chi_nb['test_accuracy']))
print('Precision NB:', np.mean(scores_chi_nb['test_precision']))
print('Recall NB:', np.mean(scores_chi_nb['test_recall']))
print('F1 NB:', np.mean(scores_chi_nb['test_f1']))
print('ROC-AUC NB:', np.mean(scores_chi_nb['test_roc_auc']))

Accuracy NB: 0.6480952380952381
Precision NB: 0.795
Recall NB: 0.6916666666666667
F1 NB: 0.6965873015873016
ROC-AUC NB: 0.6375


In [32]:
pipe_chi_rf = Pipeline([('Chi-cuadrado',feat_sel_chi), ('RF',clf_rf)])


In [33]:
%%time

scores_chi_rf = cross_validate(pipe_chi_rf, X, y, cv=10, scoring=('accuracy','precision','recall','f1', 'roc_auc'),return_train_score=True)

CPU times: user 10.5 s, sys: 708 ms, total: 11.2 s
Wall time: 12.4 s


In [34]:
print('Accuracy RF:', np.mean(scores_chi_rf['test_accuracy']))
print('Precision RF:', np.mean(scores_chi_rf['test_precision']))
print('Recall RF:', np.mean(scores_chi_rf['test_recall']))
print('F1 RF:', np.mean(scores_chi_rf['test_f1']))
print('ROC-AUC RF:', np.mean(scores_chi_rf['test_roc_auc']))

Accuracy RF: 0.7066666666666668
Precision RF: 0.76
Recall RF: 0.8666666666666666
F1 RF: 0.7915873015873015
ROC-AUC RF: 0.6791666666666667


## SELECCIONAR VARIABLES USANDO ARBOLES DE DECISION

In [35]:
pipe_dt_svc = Pipeline([('DT', SelectFromModel(clf_rf, max_features=10, threshold=-np.inf)), ('SVC',clf_svc)])


In [36]:
%%time

scores_dt_svc = cross_validate(pipe_dt_svc, X, y, cv=10, scoring=('accuracy','precision','recall','f1', 'roc_auc'),return_train_score=True)

CPU times: user 5.8 s, sys: 413 ms, total: 6.21 s
Wall time: 7.47 s


In [37]:
print('Accuracy DT:', np.mean(scores_dt_svc['test_accuracy']))
print('Precision DT:', np.mean(scores_dt_svc['test_precision']))
print('Recall DT:', np.mean(scores_dt_svc['test_recall']))
print('F1 DT:', np.mean(scores_dt_svc['test_f1']))
print('ROC-AUC DT:', np.mean(scores_dt_svc['test_roc_auc']))

Accuracy DT: 0.6004761904761904
Precision DT: 0.6504761904761904
Recall DT: 0.8416666666666666
F1 DT: 0.7254256854256854
ROC-AUC DT: 0.6083333333333333


In [38]:
pipe_dt_nb = Pipeline([('DT', SelectFromModel(clf_rf, max_features=10, threshold=-np.inf)), ('SVC',clf_nb)])


In [39]:
%%time

scores_dt_nb = cross_validate(pipe_dt_nb, X, y, cv=10, scoring=('accuracy','precision','recall','f1', 'roc_auc'),return_train_score=True)

CPU times: user 5.8 s, sys: 354 ms, total: 6.16 s
Wall time: 7.54 s


In [40]:
print('Accuracy DT:', np.mean(scores_dt_nb['test_accuracy']))
print('Precision DT:', np.mean(scores_dt_nb['test_precision']))
print('Recall DT:', np.mean(scores_dt_nb['test_recall']))
print('F1 DT:', np.mean(scores_dt_nb['test_f1']))
print('ROC-AUC DT:', np.mean(scores_dt_nb['test_roc_auc']))

Accuracy DT: 0.5671428571428571
Precision DT: 0.6854761904761906
Recall DT: 0.6916666666666667
F1 DT: 0.6658225108225108
ROC-AUC DT: 0.6416666666666667


In [41]:
pipe_dt_rf = Pipeline([('DT', SelectFromModel(clf_rf, max_features=10, threshold=-np.inf)), ('SVC',clf_rf)])


In [42]:
%%time

scores_dt_rf = cross_validate(pipe_dt_rf, X, y, cv=10, scoring=('accuracy','precision','recall','f1', 'roc_auc'),return_train_score=True)

CPU times: user 11.9 s, sys: 715 ms, total: 12.6 s
Wall time: 21.6 s


In [43]:
print('Accuracy DT:', np.mean(scores_dt_rf['test_accuracy']))
print('Precision DT:', np.mean(scores_dt_rf['test_precision']))
print('Recall DT:', np.mean(scores_dt_rf['test_recall']))
print('F1 DT:', np.mean(scores_dt_rf['test_f1']))
print('ROC-AUC DT:', np.mean(scores_dt_rf['test_roc_auc']))

Accuracy DT: 0.6004761904761904
Precision DT: 0.6599999999999999
Recall DT: 0.7666666666666667
F1 DT: 0.7012698412698414
ROC-AUC DT: 0.60625


## SELECCIONAR VARIABLES USANDO LOGISTIC REGRESSION Y L1

In [44]:
pipe_l1_svc = Pipeline([('LR', SelectFromModel(LogisticRegression(penalty='l1', solver='liblinear'), max_features=10, threshold=-np.inf)), ('SVC',clf_svc)])


In [45]:
%%time

scores_l1_svc = cross_validate(pipe_l1_svc, X, y, cv=10, scoring=('accuracy','precision','recall','f1', 'roc_auc'),return_train_score=True)

CPU times: user 325 ms, sys: 24 ms, total: 349 ms
Wall time: 348 ms


In [46]:
print('Accuracy LR:', np.mean(scores_l1_svc['test_accuracy']))
print('Precision LR:', np.mean(scores_l1_svc['test_precision']))
print('Recall LR:', np.mean(scores_l1_svc['test_recall']))
print('F1 LR:', np.mean(scores_l1_svc['test_f1']))
print('ROC-AUC LR:', np.mean(scores_l1_svc['test_roc_auc']))

Accuracy LR: 0.667142857142857
Precision LR: 0.7849999999999999
Recall LR: 0.7166666666666667
F1 LR: 0.7132539682539684
ROC-AUC LR: 0.6541666666666667


In [47]:
pipe_l1_nb = Pipeline([('LR', SelectFromModel(LogisticRegression(penalty='l1', solver='liblinear'), max_features=10, threshold=-np.inf)), ('NB',clf_nb)])


In [48]:
%%time

scores_l1_nb = cross_validate(pipe_l1_nb, X, y, cv=10, scoring=('accuracy','precision','recall','f1', 'roc_auc'),return_train_score=True)

CPU times: user 348 ms, sys: 0 ns, total: 348 ms
Wall time: 347 ms


In [49]:
print('Accuracy LR:', np.mean(scores_l1_nb['test_accuracy']))
print('Precision LR:', np.mean(scores_l1_nb['test_precision']))
print('Recall LR:', np.mean(scores_l1_nb['test_recall']))
print('F1 LR:', np.mean(scores_l1_nb['test_f1']))
print('ROC-AUC LR:', np.mean(scores_l1_nb['test_roc_auc']))

Accuracy LR: 0.628095238095238
Precision LR: 0.7566666666666666
Recall LR: 0.7166666666666667
F1 LR: 0.6993650793650794
ROC-AUC LR: 0.7250000000000001


In [50]:
pipe_l1_rf = Pipeline([('LR', SelectFromModel(LogisticRegression(penalty='l1', solver='liblinear'), max_features=10, threshold=-np.inf)), ('RF',clf_rf)])


In [51]:
%%time

scores_l1_rf = cross_validate(pipe_l1_rf, X, y, cv=10, scoring=('accuracy','precision','recall','f1', 'roc_auc'),return_train_score=True)

CPU times: user 5.05 s, sys: 694 ms, total: 5.75 s
Wall time: 12.4 s


In [52]:
print('Accuracy LR:', np.mean(scores_l1_rf['test_accuracy']))
print('Precision LR:', np.mean(scores_l1_rf['test_precision']))
print('Recall LR:', np.mean(scores_l1_rf['test_recall']))
print('F1 LR:', np.mean(scores_l1_rf['test_f1']))
print('ROC-AUC LR:', np.mean(scores_l1_rf['test_roc_auc']))

Accuracy LR: 0.6780952380952382
Precision LR: 0.7316666666666667
Recall LR: 0.8666666666666666
F1 LR: 0.7588095238095238
ROC-AUC LR: 0.6145833333333334


In [53]:
dill.dump_session('CNS-CV.db')


In [54]:
dill.load_session('CNS-CV.db')
